In [8]:
import requests
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import GRU, Dense, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import joblib
import time
import os

In [9]:

# ## Fetch Crypto Data

def fetch_crypto_data(symbol, interval, limit=100, retries=3, delay=2):
    url = f"https://api.binance.us/api/v3/klines?symbol={symbol}USDT&interval={interval}&limit={limit}"
    for attempt in range(retries):
        try:
            response = requests.get(url)
            response.raise_for_status()
            data = response.json()
            df = pd.DataFrame(data, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time',
                                             'quote_asset_volume', 'trades', 'taker_buy_base', 'taker_buy_quote', 'ignore'])
            df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
            df.set_index('timestamp', inplace=True)
            df['close'] = df['close'].astype(float)
            return df[['close']]
        except requests.exceptions.RequestException as e:
            print(f"API Error (Attempt {attempt + 1}/{retries}): {e}")
            time.sleep(delay)
    print("Failed to fetch data after retries.")
    return None

In [10]:
# ## Prepare Data

# %%
def prepare_data(df, time_steps=20):
    if len(df) <= time_steps:
        raise ValueError(f"Dataset too small for time_steps={time_steps}. Needs at least {time_steps + 1} rows.")
    
    scaler = MinMaxScaler()
    df_scaled = scaler.fit_transform(df)
    
    X, y = [], []
    for i in range(len(df_scaled) - time_steps):
        X.append(df_scaled[i:i + time_steps])
        y.append(df_scaled[i + time_steps])
    
    X, y = np.array(X), np.array(y)
    return X, y, scaler

In [11]:
# ## Build GRU Model

# %%
def build_gru_model(input_shape):
    model = Sequential([
        GRU(50, return_sequences=True, input_shape=input_shape, kernel_regularizer=l2(0.01)),
        Dropout(0.2),
        GRU(50, return_sequences=False, kernel_regularizer=l2(0.01)),
        Dropout(0.2),
        Dense(25),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [12]:
def train_and_save_model(symbol, interval, epochs=100, batch_size=16, time_steps=20, model_filename="crypto_model.h5"):
    df = fetch_crypto_data(symbol, interval)
    if df is None:
        return None
    
    X, y, scaler = prepare_data(df, time_steps)
    train_size = int(len(X) * 0.8)
    X_train, y_train = X[:train_size], y[:train_size]
    X_test, y_test = X[train_size:], y[train_size:]
    
    model = build_gru_model((X.shape[1], X.shape[2]))
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    
    # Add model checkpoint callback to save the best model
    model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
        model_filename, monitor='val_loss', save_best_only=True, mode='min', verbose=1
    )
    
    model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=batch_size,
              callbacks=[early_stopping, model_checkpoint])
    
    # Load the best model after training finishes
    model = load_model(model_filename)
    
    model.save(model_filename)
    scaler_filename = model_filename.replace(".h5", "_scaler.pkl")
    joblib.dump(scaler, scaler_filename)
    print(f"Model saved as {model_filename}, Scaler saved as {scaler_filename}")
    return model_filename, scaler_filename


In [13]:
# ## Evaluate Model

# %%
def evaluate_model(model, X_test, y_test, scaler):
    predictions = model.predict(X_test)
    predictions = scaler.inverse_transform(predictions)
    y_test_original = scaler.inverse_transform(y_test)
    
    rmse = np.sqrt(mean_squared_error(y_test_original, predictions))
    mae = mean_absolute_error(y_test_original, predictions)
    
    print(f"RMSE: {rmse}")
    print(f"MAE: {mae}")


In [14]:
# ## Load Model and Predict

# %%
def load_and_predict(model_filename, symbol, interval, time_steps=20):
    df = fetch_crypto_data(symbol, interval)
    if df is None:
        return None
    
    X, _, scaler = prepare_data(df, time_steps)
    model = load_model(model_filename)
    predictions = model.predict(X)
    predictions = scaler.inverse_transform(predictions)
    
    return df.index[-len(predictions):], predictions